In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 네이버 로그인 페이지로 이동
driver.get('https://nid.naver.com/nidlogin.login')

# 수동 로그인 안내 메시지
input("네이버에 로그인한 후 Enter 키를 눌러주세요...")

# 네이버 카페 '수혜' 페이지로 이동
cafe_url = "https://cafe.naver.com/suhui"
driver.get(cafe_url)

# 페이지 로드 대기
time.sleep(3)

# '이주의 인기글 모음' 카테고리 클릭
popular_post_xpath = '/html/body/div[3]/div/div[6]/div[1]/div[3]/div[2]/ul[2]/li[1]/a'
popular_category = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, popular_post_xpath)))
popular_category.click()

# iframe으로 전환
time.sleep(3)
driver.switch_to.frame("cafe_main")

# 충분히 스크롤을 내려 본문이 보이도록 설정
def scroll_down(driver, scroll_count=5):
    for _ in range(scroll_count):
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        time.sleep(1)

# 스크롤을 5번 반복하여 본문까지 로드
scroll_down(driver, scroll_count=5)

# 게시글 데이터를 저장할 리스트
data = []

# 페이지에서 게시글 정보 수집 함수
def get_articles_from_page(driver):
    time.sleep(2)
    
    # 현재 페이지의 게시글 크롤링
    articles = driver.find_elements(By.CSS_SELECTOR, '.article-board tbody tr')
    for article in articles:
        try:
            # 제목
            title_elem = article.find_elements(By.CSS_SELECTOR, '.board-list .article')
            title = title_elem[0].text.strip() if title_elem else "N/A"

            # 작성자
            writer_elem = article.find_elements(By.CSS_SELECTOR, '.td_name .p-nick .m-tcol-c')
            writer = writer_elem[0].text.strip() if writer_elem else "N/A"

            # 작성일
            date_elem = article.find_elements(By.CLASS_NAME, 'td_date')
            date = date_elem[0].text.strip() if date_elem else "N/A"

            # 조회수
            views_elem = article.find_elements(By.CLASS_NAME, 'td_view')
            views = views_elem[0].text.strip().replace(",", "") if views_elem else "N/A"

            # 좋아요
            likes_elem = article.find_elements(By.CLASS_NAME, 'td_likes')
            likes = likes_elem[0].text.strip().replace(",", "") if likes_elem else "N/A"
            
            # 게시글 정보를 리스트에 추가
            data.append({
                "제목": title,
                "작성자": writer,
                "작성일": date,
                "조회수": views,
                "좋아요": likes
            })
            
            # 수집된 게시글 정보를 출력
            print(f"제목: {title}, 작성자: {writer}, 작성일: {date}, 조회수: {views}, 좋아요: {likes}")
            
        except Exception as e:
            print("데이터 수집 중 오류:", e)

# 첫 페이지 크롤링
get_articles_from_page(driver)

# 다음 페이지로 이동하는 루프
while True:
    try:
        # "다음" 버튼 클릭
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='pgR']")))
        next_button.click()
        
        # 다음 페이지 게시글 크롤링
        scroll_down(driver, scroll_count=5)  # 다음 페이지에서도 충분히 스크롤
        get_articles_from_page(driver)
    except:
        print("더 이상 다음 페이지가 없습니다.")
        break

# 브라우저 종료
driver.quit()

# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(data)
df.to_csv("popular_articles.csv", index=False, encoding='utf-8-sig')
print("CSV 파일로 저장이 완료되었습니다. (popular_articles.csv)")

제목: 수능 채점 이벤트+교육 이벤트 모음-11.13 update, 작성자: 수만휘, 작성일: 2020.03.13., 조회수: 69만, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음, 작성자: 수만휘, 작성일: 2020.03.13., 조회수: 84만, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 D-1 수능 관련 Q&A 90개 총정리 (제가 알고있는건 다 적어봤습니다), 작성자: 알파카, 작성일: 2024.11.13., 조회수: 7960, 좋아요: N/A
제목: N/A, 작성자: 알파카, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 D-1: 마지막 1%의 기적을 만드는 법!! [수능시간대별 행동요령 꼭 체크요망], 작성자: 스밴, 작성일: 2024.11.13., 조회수: 670, 좋아요: N/A
제목: N/A, 작성자: 스밴, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: [평가원 요청] 시험 종료 시간 전에 문제나 지문 유출은 주의해 주시면 감사하겠습니다., 작성자: 수만휘, 작성일: 2023.11.16., 조회수: 3151, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 유경험자 엄마의 수능 전날 팁(재업), 작성자: 다시 살고 싶은 삶, 작성일: 2024.11.12., 조회수: 7093, 좋아요: N/A
제목: N/A, 작성자: 다시 살고 싶은 삶, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: [자료] 지구과학I 수능 직전 평가원 필수 기출 정리입니다., 작성자: 세페이드 멘토, 작성일: 2024.11.12., 조회수: 802, 좋아요: N/A
제목: N/A, 작성자: 세페이드 멘토, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 괜찮아 아들

In [4]:
import pandas as pd

df = pd.read_csv('popular_articles.csv')

In [ ]:
df.tail(50)

,제목,작성자,작성일,조회수,좋아요
0,수능 채점 이벤트+교육 이벤트 모음-11.13 update,수만휘,2020.03.13.,69만,NaN
1,NaN,수만휘,NaN,NaN,NaN
2,수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음,수만휘,2020.03.13.,84만,NaN
3,NaN,수만휘,NaN,NaN,NaN
4,수능 D-1 수능 관련 Q&A 90개 총정리 (제가 알고있는건 다 적어봤습니다),알파카,2024.11.13.,7960,NaN
5,NaN,알파카,NaN,NaN,NaN
6,수능 D-1: 마지막 1%의 기적을 만드는 법!! [수능시간대별 행동요령 꼭 체크요망],스밴,2024.11.13.,670,NaN
7,NaN,스밴,NaN,NaN,NaN
8,[평가원 요청] 시험 종료 시간 전에 문제나 지문 유출은 주의해 주시면 감사하겠습니다.,수만휘,2023.11.16.,3151,NaN
9,NaN,수만휘,NaN,NaN,NaN


In [8]:
df['작성자'].nunique()

1626

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 네이버 로그인 페이지로 이동
driver.get('https://nid.naver.com/nidlogin.login')

# 수동 로그인 안내 메시지
input("네이버에 로그인한 후 Enter 키를 눌러주세요...")

# 네이버 카페 '수혜' 페이지로 이동
cafe_url = "https://cafe.naver.com/suhui"
driver.get(cafe_url)

# 페이지 로드 대기
time.sleep(3)

# '이주의 인기글 모음' 카테고리 클릭
popular_post_xpath = '/html/body/div[3]/div/div[6]/div[1]/div[3]/div[2]/ul[2]/li[1]/a'
popular_category = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, popular_post_xpath)))
popular_category.click()

# iframe으로 전환
time.sleep(3)
driver.switch_to.frame("cafe_main")

# 충분히 스크롤을 내려 본문이 보이도록 설정
def scroll_down(driver, scroll_count=5):
    for _ in range(scroll_count):
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        time.sleep(1)

# 스크롤을 5번 반복하여 본문까지 로드
scroll_down(driver, scroll_count=5)

# 게시글 데이터를 저장할 리스트
data = []

def get_articles_from_page(driver):
    time.sleep(2)
    
    # 게시글 테이블의 각 행을 선택
    articles = driver.find_elements(By.CSS_SELECTOR, '.article-board tbody tr')
    for article in articles:
        try:
            # 제목 추출
            title_elem = article.find_elements(By.CSS_SELECTOR, '.board-list .article')
            title = title_elem[0].text.strip() if title_elem else "N/A"

            # 작성자 추출
            writer_elem = article.find_elements(By.CSS_SELECTOR, '.td_name .p-nick .m-tcol-c')
            writer = writer_elem[0].text.strip() if writer_elem else "N/A"

            # 작성일 추출
            date_elem = article.find_elements(By.CLASS_NAME, 'td_date')
            date = date_elem[0].text.strip() if date_elem else "N/A"

            # 조회수 추출 (필요한 경우 XPath 추가)
            views_elem = article.find_elements(By.CLASS_NAME, 'td_view')
            views = views_elem[0].text.strip().replace(",", "") if views_elem else "N/A"

            # 좋아요 추출 (XPath로 예비 선택자 추가)
            likes_elem = article.find_elements(By.CLASS_NAME, 'td_likes') or article.find_elements(By.XPATH, '//*[contains(@class, "td_likes")]')
            likes = likes_elem[0].text.strip().replace(",", "") if likes_elem else "N/A"
            
            # 수집된 데이터 추가
            data.append({
                "제목": title,
                "작성자": writer,
                "작성일": date,
                "조회수": views,
                "좋아요": likes
            })
            
            # 데이터 출력
            print(f"제목: {title}, 작성자: {writer}, 작성일: {date}, 조회수: {views}, 좋아요: {likes}")
            
        except Exception as e:
            print("데이터 수집 중 오류:", e)

# 첫 페이지 크롤링
get_articles_from_page(driver)

# 다음 페이지로 이동하는 루프
while True:
    try:
        # "다음" 버튼 클릭
        next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='pgR']")))
        next_button.click()
        
        # 다음 페이지 게시글 크롤링
        scroll_down(driver, scroll_count=5)  # 다음 페이지에서도 충분히 스크롤
        get_articles_from_page(driver)
    except:
        print("더 이상 다음 페이지가 없습니다.")
        break

# 브라우저 종료
driver.quit()

# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(data)
df.to_csv("popular_articles2.csv", index=False, encoding='utf-8-sig')
print("CSV 파일로 저장이 완료되었습니다. (popular_articles2.csv)")

제목: 수능 채점 이벤트+교육 이벤트 모음-11.13 update, 작성자: 수만휘, 작성일: 2020.03.13., 조회수: 69만, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음, 작성자: 수만휘, 작성일: 2020.03.13., 조회수: 84만, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 D-1 수능 관련 Q&A 90개 총정리 (제가 알고있는건 다 적어봤습니다), 작성자: 알파카, 작성일: 2024.11.13., 조회수: 7991, 좋아요: N/A
제목: N/A, 작성자: 알파카, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 D-1: 마지막 1%의 기적을 만드는 법!! [수능시간대별 행동요령 꼭 체크요망], 작성자: 스밴, 작성일: 2024.11.13., 조회수: 677, 좋아요: N/A
제목: N/A, 작성자: 스밴, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: [평가원 요청] 시험 종료 시간 전에 문제나 지문 유출은 주의해 주시면 감사하겠습니다., 작성자: 수만휘, 작성일: 2023.11.16., 조회수: 3152, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 유경험자 엄마의 수능 전날 팁(재업), 작성자: 다시 살고 싶은 삶, 작성일: 2024.11.12., 조회수: 7105, 좋아요: N/A
제목: N/A, 작성자: 다시 살고 싶은 삶, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: [자료] 지구과학I 수능 직전 평가원 필수 기출 정리입니다., 작성자: 세페이드 멘토, 작성일: 2024.11.12., 조회수: 803, 좋아요: N/A
제목: N/A, 작성자: 세페이드 멘토, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 괜찮아 아들

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 네이버 로그인 페이지로 이동
driver.get('https://nid.naver.com/nidlogin.login')

# 수동 로그인 안내 메시지
input("네이버에 로그인한 후 Enter 키를 눌러주세요...")

# 네이버 카페 '수혜' 페이지로 이동
cafe_url = "https://cafe.naver.com/suhui"
driver.get(cafe_url)

# 페이지 로드 대기
time.sleep(3)

# '이주의 인기글 모음' 카테고리 클릭
popular_post_xpath = '/html/body/div[3]/div/div[6]/div[1]/div[3]/div[2]/ul[2]/li[1]/a'
popular_category = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, popular_post_xpath)))
popular_category.click()

# iframe으로 전환
time.sleep(3)
driver.switch_to.frame("cafe_main")

# 충분히 스크롤을 내려 본문이 보이도록 설정
def scroll_down(driver, scroll_count=5):
    for _ in range(scroll_count):
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        time.sleep(1)

# 게시글 데이터를 저장할 리스트
data = []

def get_articles_from_page(driver):
    time.sleep(2)
    
    # 게시글 테이블의 각 행을 선택
    articles = driver.find_elements(By.CSS_SELECTOR, '.article-board tbody tr')
    for article in articles:
        try:
            # 제목 추출
            title_elem = article.find_elements(By.CSS_SELECTOR, '.board-list .article')
            title = title_elem[0].text.strip() if title_elem else "N/A"

            # 작성자 추출
            writer_elem = article.find_elements(By.CSS_SELECTOR, '.td_name .p-nick .m-tcol-c')
            writer = writer_elem[0].text.strip() if writer_elem else "N/A"

            # 작성일 추출
            date_elem = article.find_elements(By.CLASS_NAME, 'td_date')
            date = date_elem[0].text.strip() if date_elem else "N/A"

            # 조회수 추출
            views_elem = article.find_elements(By.CLASS_NAME, 'td_view')
            views = views_elem[0].text.strip().replace(",", "") if views_elem else "N/A"

            # 좋아요 추출 (XPath 예비 선택자 사용)
            likes_elem = article.find_elements(By.CLASS_NAME, 'td_likes')
            if not likes_elem:  # 요소가 없으면 다른 선택자를 사용해 시도
                likes_elem = article.find_elements(By.XPATH, '//*[contains(@class, "td_likes") or contains(@class, "likes")]')
            likes = likes_elem[0].text.strip().replace(",", "") if likes_elem else "N/A"
            
            # 수집된 데이터 추가
            data.append({
                "제목": title,
                "작성자": writer,
                "작성일": date,
                "조회수": views,
                "좋아요": likes
            })
            
            # 데이터 출력
            print(f"제목: {title}, 작성자: {writer}, 작성일: {date}, 조회수: {views}, 좋아요: {likes}")
            
        except Exception as e:
            print("데이터 수집 중 오류:", e)

# 페이지 순회와 네비게이션
def navigate_pages(driver):
    current_page = 1
    while True:
        # 현재 페이지에서 게시글 크롤링
        get_articles_from_page(driver)
        
        try:
            # 현재 페이지 번호 찾기
            page_numbers = driver.find_elements(By.CSS_SELECTOR, ".prev-next a")
            page_texts = [page.text for page in page_numbers]
            
            # 현재 페이지가 마지막 페이지 번호일 때 다음 버튼 클릭
            if str(current_page) in page_texts:
                page_numbers[page_texts.index(str(current_page))].click()
                current_page += 1
                time.sleep(2)  # 페이지가 로드될 시간을 주기
            else:
                # 다음 버튼 클릭
                next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='pgR']")))
                next_button.click()
                current_page = 1  # 페이지 번호를 다시 시작
                time.sleep(2)
                
        except:
            print("더 이상 다음 페이지가 없습니다.")
            break

# 첫 페이지부터 모든 페이지 순회
navigate_pages(driver)

# 브라우저 종료
driver.quit()

# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(data)
df.to_csv("popular_articles2.csv", index=False, encoding='utf-8-sig')
print("CSV 파일로 저장이 완료되었습니다. (popular_articles2.csv)")

제목: 수능 채점 이벤트+교육 이벤트 모음-11.13 update, 작성자: 수만휘, 작성일: 2020.03.13., 조회수: 69만, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음, 작성자: 수만휘, 작성일: 2020.03.13., 조회수: 84만, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 D-1 수능 관련 Q&A 90개 총정리 (제가 알고있는건 다 적어봤습니다), 작성자: 알파카, 작성일: 2024.11.13., 조회수: 7997, 좋아요: N/A
제목: N/A, 작성자: 알파카, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 D-1: 마지막 1%의 기적을 만드는 법!! [수능시간대별 행동요령 꼭 체크요망], 작성자: 스밴, 작성일: 2024.11.13., 조회수: 678, 좋아요: N/A
제목: N/A, 작성자: 스밴, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: [평가원 요청] 시험 종료 시간 전에 문제나 지문 유출은 주의해 주시면 감사하겠습니다., 작성자: 수만휘, 작성일: 2023.11.16., 조회수: 3152, 좋아요: N/A
제목: N/A, 작성자: 수만휘, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 수능 유경험자 엄마의 수능 전날 팁(재업), 작성자: 다시 살고 싶은 삶, 작성일: 2024.11.12., 조회수: 7105, 좋아요: N/A
제목: N/A, 작성자: 다시 살고 싶은 삶, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: [자료] 지구과학I 수능 직전 평가원 필수 기출 정리입니다., 작성자: 세페이드 멘토, 작성일: 2024.11.12., 조회수: 804, 좋아요: N/A
제목: N/A, 작성자: 세페이드 멘토, 작성일: N/A, 조회수: N/A, 좋아요: N/A
제목: 괜찮아 아들

In [12]:
df2 = pd.read_csv('popular_articles2.csv')

In [13]:
df2

,제목,작성자,작성일,조회수,좋아요
0,수능 채점 이벤트+교육 이벤트 모음-11.13 update,수만휘,2020.03.13.,69만,NaN
1,NaN,수만휘,NaN,NaN,NaN
2,수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음,수만휘,2020.03.13.,84만,NaN
3,NaN,수만휘,NaN,NaN,NaN
4,수능 D-1 수능 관련 Q&A 90개 총정리 (제가 알고있는건 다 적어봤습니다),알파카,2024.11.13.,7997,NaN
...,...,...,...,...,...
8492,NaN,ivcnawiltl,NaN,NaN,NaN
8493,동국대 점공 1등 광탈,이라라랑,2024.11.12.,1007,NaN
8494,NaN,이라라랑,NaN,NaN,NaN
8495,면접에서 성적 하락 원인 물어보면,축하드려요합격입니다,2024.11.12.,262,NaN


In [16]:
df2[df2['제목'] == '[미대 정시] 탐구']

,제목,작성자,작성일,조회수,좋아요
1701,[미대 정시] 탐구,sjidwiw,2024.11.12.,107,NaN
5075,[미대 정시] 탐구,sjidwiw,2024.11.12.,107,NaN
8447,[미대 정시] 탐구,sjidwiw,2024.11.12.,107,NaN


In [20]:
df2_cleaned = df2.dropna(subset=['제목', '작성자', '작성일', '조회수'])

In [21]:
df2_cleaned

,제목,작성자,작성일,조회수,좋아요
0,수능 채점 이벤트+교육 이벤트 모음-11.13 update,수만휘,2020.03.13.,69만,NaN
2,수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음,수만휘,2020.03.13.,84만,NaN
4,수능 D-1 수능 관련 Q&A 90개 총정리 (제가 알고있는건 다 적어봤습니다),알파카,2024.11.13.,7997,NaN
6,수능 D-1: 마지막 1%의 기적을 만드는 법!! [수능시간대별 행동요령 꼭 체크요망],스밴,2024.11.13.,678,NaN
8,[평가원 요청] 시험 종료 시간 전에 문제나 지문 유출은 주의해 주시면 감사하겠습니다.,수만휘,2023.11.16.,3152,NaN
...,...,...,...,...,...
8487,"수액 맞을까요,,?",춘식이이모,2024.11.12.,217,NaN
8489,서영대 파주캠 간호 예비 37번 가능성 있나요..,다 떨어지게 생겼네,2024.11.12.,153,NaN
8491,"쌍사 수특, 수완",ivcnawiltl,2024.11.12.,71,NaN
8493,동국대 점공 1등 광탈,이라라랑,2024.11.12.,1007,NaN


In [22]:
# 전체 행에서 중복 제거 (모든 열이 동일할 때 중복으로 간주)
df2_cleaned = df2_cleaned.drop_duplicates()

In [23]:
df2_cleaned

,제목,작성자,작성일,조회수,좋아요
0,수능 채점 이벤트+교육 이벤트 모음-11.13 update,수만휘,2020.03.13.,69만,NaN
2,수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음,수만휘,2020.03.13.,84만,NaN
4,수능 D-1 수능 관련 Q&A 90개 총정리 (제가 알고있는건 다 적어봤습니다),알파카,2024.11.13.,7997,NaN
6,수능 D-1: 마지막 1%의 기적을 만드는 법!! [수능시간대별 행동요령 꼭 체크요망],스밴,2024.11.13.,678,NaN
8,[평가원 요청] 시험 종료 시간 전에 문제나 지문 유출은 주의해 주시면 감사하겠습니다.,수만휘,2023.11.16.,3152,NaN
...,...,...,...,...,...
8349,제 자신이 너무 한심하네요,아카락카,2024.11.12.,405,NaN
8369,서울여대 지원하신 학생 분들께 알립니다.,돌보슈,2024.11.12.,1482,NaN
8379,아주대 전자공학,6종합생기부러,2024.11.12.,760,NaN
8453,여대 공학 전환 반대하는 이유가 뭐임? 망해가는 잡대 살리려면 어쩔수 없는건데,시대인재의개,2024.11.12.,1527,NaN


In [24]:
# 조회수 컬럼을 변환하는 함수
def convert_views(view):
    if '만' in view:
        # "만"을 제거하고 float으로 변환한 후 10000을 곱함
        return int(float(view.replace('만', '')) * 10000)
    else:
        # 만 단위가 없는 경우는 그냥 정수로 변환
        return int(view)

# 조회수 컬럼에 함수 적용
df2_cleaned['조회수'] = df2_cleaned['조회수'].apply(convert_views)

/var/folders/lb/80g3vygs31z9yblw4jht2q4m0000gn/T/ipykernel_77491/1802753555.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_cleaned['조회수'] = df2_cleaned['조회수'].apply(convert_views)


In [25]:
df2_cleaned

,제목,작성자,작성일,조회수,좋아요
0,수능 채점 이벤트+교육 이벤트 모음-11.13 update,수만휘,2020.03.13.,690000,NaN
2,수능 유의사항과 가채점표 파일 배포+자료 및 좋은 글 모음,수만휘,2020.03.13.,840000,NaN
4,수능 D-1 수능 관련 Q&A 90개 총정리 (제가 알고있는건 다 적어봤습니다),알파카,2024.11.13.,7997,NaN
6,수능 D-1: 마지막 1%의 기적을 만드는 법!! [수능시간대별 행동요령 꼭 체크요망],스밴,2024.11.13.,678,NaN
8,[평가원 요청] 시험 종료 시간 전에 문제나 지문 유출은 주의해 주시면 감사하겠습니다.,수만휘,2023.11.16.,3152,NaN
...,...,...,...,...,...
8349,제 자신이 너무 한심하네요,아카락카,2024.11.12.,405,NaN
8369,서울여대 지원하신 학생 분들께 알립니다.,돌보슈,2024.11.12.,1482,NaN
8379,아주대 전자공학,6종합생기부러,2024.11.12.,760,NaN
8453,여대 공학 전환 반대하는 이유가 뭐임? 망해가는 잡대 살리려면 어쩔수 없는건데,시대인재의개,2024.11.12.,1527,NaN


In [26]:
df2_cleaned.dtypes

제목      object
작성자     object
작성일     object
조회수      int64
좋아요    float64
dtype: object